In [1]:
# Hugging Face Embedding
# %pip install --upgrade --quiet  langchain sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # 허깅페이스 API 토큰 얻어옴.
print(f"[API KEY]\n{os.environ['HUGGINGFACEHUB_API_TOKEN']}")


In [9]:
HuggingFaceEmbeddings?

Init signature:
HuggingFaceEmbeddings(
    *,
    client: Any = None,
    model_name: str = 'sentence-transformers/all-mpnet-base-v2',
    cache_folder: Optional[str] = None,
    model_kwargs: Dict[str, Any] = None,
    encode_kwargs: Dict[str, Any] = None,
    multi_process: bool = False,
    show_progress: bool = False,
) -> None
Docstring:     
HuggingFace sentence_transformers embedding models.

To use, you should have the ``sentence_transformers`` python package installed.

Example:
    .. code-block:: python

        from langchain_community.embeddings import HuggingFaceEmbeddings

        model_name = "sentence-transformers/all-mpnet-base-v2"
        model_kwargs = {'device': 'cpu'}
        encode_kwargs = {'normalize_embeddings': False}
        hf = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
        )
Init docstring: Initialize the sentence_transformer.
File:           /usr/local/lib/p

In [15]:
# ./cache/ 경로에 다운로드 받도록 설정
os.environ["HF_HOME"] = "./cache/"

from langchain_community.embeddings import (
    HuggingFaceEmbeddings,
    HuggingFaceBgeEmbeddings,
)

# model_name = 없으면: 기본 'sentence-transformers/all-mpnet-base-v2' 모델임.
#embeddings = HuggingFaceEmbeddings()  # HuggingFace 임베딩을 생성합니다.
embeddings = HuggingFaceEmbeddings(
    model_name="bongsoo/kpf-sbert-128d-v1", 
    show_progress=True
)

print(f'*임베딩 모델: {embeddings.model_name}')

*임베딩 모델: bongsoo/kpf-sbert-128d-v1


In [16]:
text = "임베딩 테스트 문장입니다."

query_result = embeddings.embed_query(text)

print(f'*벡터길이: {len(query_result)}')
print(f'*벡터: {query_result[:10]}')

Batches: 100%|██████████| 1/1 [00:00<00:00, 22.08it/s]

*벡터길이: 128
*벡터: [-0.20591969788074493, 0.2096341997385025, 0.00921131856739521, -0.007399042136967182, -0.1793106496334076, 0.1496046781539917, 0.2164066582918167, 0.2643868029117584, 0.15830323100090027, -0.4097329080104828]


In [19]:
# Faiss와 CacheBackedEmbeddings 이용해서 문서를 임베딩 하는 예시

from langchain.storage import LocalFileStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings

# 로컬 파일 저장소 설정
# => cache 폴더에 가보면, text-embedding-ada-002b41bd27f-7cf5-5323-bc34-d315497941a6 식으로 각 임베딩 값이 만들어저 있음.
store = LocalFileStore("./cache/")
#store = InMemoryByteStore()  # 메모리 내 바이트 저장소 생성

# 캐시 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    store,
    namespace=embeddings.model_name, # 네임스페이스로 임베딩 모델명 지정
)

# RecursiveCharacterTextSplitter 로 문서를 불러와서 chunk 함.
with open("../data/04.급여규정_16.06.01.txt") as f:
    file = f.read()

print(f'*type:{type(file)}')
print(f'*file:\n{file[:100]}\n')

text_splitter = RecursiveCharacterTextSplitter(
    # 청크 크기를 매우 작게 설정합니다. 예시를 위한 설정입니다.
    chunk_size=250,
    # 청크 간의 중복되는 문자 수를 설정합니다.
    chunk_overlap=50,
    # 문자열 길이를 계산하는 함수를 지정합니다.
    length_function=len,
    # 구분자로 정규식을 사용할지 여부를 설정합니다.
    is_separator_regex=False,
)

# text_splitter 이용해서 file 텍스트를 문서단위로 출력
# -> separator를 지정하지 않았으므로, 문서에서 기본값 "\n\n" 로 구분해서 나눔.
documents = text_splitter.create_documents([file])
print(f'*문서 분할 수: {len(documents)}')

for document in documents:
    print(document)
    print(f'--' * 20)
    

*type:<class 'str'>
*file:
급여규정
1. 목적
이 규정은 주식회사 모코엠시스에 근무하는 직원의 급여에 관한 제반사항을 정하여 합리적인 급여관리를 실행함을 목적으로 한다.
2. 적용범위
직원의 급여는 관계법령

*문서 분할 수: 17
page_content='급여규정\n1. 목적\n이 규정은 주식회사 모코엠시스에 근무하는 직원의 급여에 관한 제반사항을 정하여 합리적인 급여관리를 실행함을 목적으로 한다.\n2. 적용범위\n직원의 급여는 관계법령, 기타 별도로 정한 것을 제외하고는 이 규정이 정하는 바에 따른다.\n3. 급여의 구성\n3.1 급여는 기본급, 제수당, 상여금, 퇴직금으로 구분한다.\n3.2 일반적으로 월급여라 함은 기본급과 제수당을 합한 금액을 말한다.\n4. 급여조정'
----------------------------------------
page_content='3.2 일반적으로 월급여라 함은 기본급과 제수당을 합한 금액을 말한다.\n4. 급여조정\n급여의 인상은 년 1회 1월 1일부로 실시함을 원칙으로 한다.\n5. 계산과 지급\n5.1 계산기간\n급여계산기간은 매월 1일부터 당월 말일까지로 한다.\n5.2 일할계산\n5.2.1 급여의 변경이 있는 월의 급여는 일할계산한다.\n5.2.2 일할계산은 월의 대소에 관계없이 월급여의 1/30로 한다.\n5.3 일할일수'
----------------------------------------
page_content='5.2.2 일할계산은 월의 대소에 관계없이 월급여의 1/30로 한다.\n5.3 일할일수\n일할계산에 있어 그 일할일수는 실근로일수를 기준으로 한다.\n5.4 계산단위\n급여계산은 원단위에서 이를 절사한다.\n5.5 급여지급일\n5.5.1 급여지급일은 매월 20일로 한다.\n단, 지급일이 휴일인 경우에는 그 전일로 한다.\n5.5.2 비상재해, 기타 부득이한 사유가 발생시에는 지급일을 변경할 수 있다\n5.6 지급방법'
---------------------------

In [20]:
# 코드 실행 시간을 측정합니다.
%time db = FAISS.from_documents(documents, cached_embedder)  # 문서로부터 FAISS 데이터베이스 생성

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.64s/it]


CPU times: user 13 s, sys: 0 ns, total: 13 s
Wall time: 1.99 s


In [21]:
# 캐싱된 임베딩을 사용하여 FAISS 데이터베이스 생성
# => **벡터 저장소를 다시 생성하려고 하면, 임베딩을 다시 계산할 필요가 없기 때문에 훨씬 더 빠르게 처리됩니다.
%time db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: user 18.3 ms, sys: 0 ns, total: 18.3 ms
Wall time: 240 ms


In [22]:
# Hugging Face Inference API
# => 허깅페이스 Inference API를 통해 모델어 접근할수 도 있다.
# => 이때는 모델을 다운로드 받거나 설치할 필요없이 호스팅되는 서버에 접근해서 실행된다.

from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

inference_api_key = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key = inference_api_key,
    model_name = "bongsoo/kpf-sbert-128d-v1",
)

# 임베딩 해봄
text = "임베딩 테스트 문장입니다."

query_result = embeddings.embed_query(text)

print(f'*벡터길이: {len(query_result)}')
print(f'*벡터: {query_result[:10]}')

*벡터길이: 128
*벡터: [-0.2059198021888733, 0.20963433384895325, 0.009211340919137001, -0.007399368099868298, -0.17931078374385834, 0.14960499107837677, 0.21640677750110626, 0.26438668370246887, 0.15830320119857788, -0.40973299741744995]


In [28]:
# Hugging Face Hub 이용
# !pip install huggingface_hub

from langchain_community.embeddings import HuggingFaceHubEmbeddings

embeddings = HuggingFaceHubEmbeddings(model="bongsoo/kpf-sbert-128d-v1")

# 임베딩 해봄
text = "임베딩 테스트 문장입니다."

query_result = embeddings.embed_query(text)

print(f'*벡터길이: {len(query_result)}')
print(f'*벡터: {query_result[:10]}')

*벡터길이: 128
*벡터: [-0.2059198021888733, 0.20963433384895325, 0.009211340919137001, -0.007399368099868298, -0.17931078374385834, 0.14960499107837677, 0.21640677750110626, 0.26438668370246887, 0.15830320119857788, -0.40973299741744995]
